## Length of the code {-}
No restriction

**Delete this section from the report, when using this template.** 

## Data quality check / cleaning / preparation 

Put code with comments. The comments should explain the code such that it can be easily understood. You may put text *(in a markdown cell)* before a large chunk of code to explain the overall purpose of the code, if it is not intuitive. **Put the name of the person / persons who contributed to each code chunk / set of code chunks.** An example is given below.

In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as sm
import seaborn as sns
import matplotlib.pyplot as plt

### Data quality check
*By Elton John*

The code below visualizes the distribution of all the variables in the dataset, and their association with the response.

In [7]:
#...Distribution of continuous variables...#

In [8]:
#...Distribution of categorical variables...#

In [9]:
#...Association of the response with the predictors...#

### Data cleaning
*By Peggy Han*

For data cleaning, we performed the following:

1. Some of the columns only have NaN values, so we removed all these variables to make the data set simpler. We also dropped some irrelevant variables that wouldn't help with developing the model.

2. In the second dataset from obtained from College Scorecard, there are about 3000 different variables. We manually selected some variables that might be relevant to be response based on the variable description.

3. When merging the two data sets, we found that some of the rows are duplicated in the resulting data frame because more than one insitution in the second dataset has the same school code, and some schools have different codes in the two data sets, so they have missing columns from the second data set. We did some manual deletion and filling in of the merged data.

4. We identified that some of the institutions were unranked in the Kaggle data set, so we dropped those rows. We also dropped columns with only 1 unique value as they do not provide insight for building the model.

5. Some of the variables from the Collge Scorecard data contains the same information are separately stored into private and public columns. For private schools, the information is stored in the private column and the public column is NaN, and vice versa. We combined these columns to use the variable as a predictor with no NaN values in the column.

6. Both data sets contain the information for average SAT score, enrollment, and admission rate information. We think the data from College Scorecard is more accurate, so we prioritize the College Scorecard data and use the kaggle data to fill in some of the missing values in the College Scorecard column to minimize the number of missing values in the variables.

The code below implements the above cleaning.

In [2]:
df1 = pd.read_json('project_data/schoolInfo.json')

In [3]:
# Dropped columns which only have NaN values
df1.dropna(axis=1, how='all', inplace=True)

# Dropped irrelevant variables
df1 = df1.drop(['nonResponderText', 'nonResponder', 'primaryPhoto', 'primaryPhotoThumb', 'aliasNames', 'urlName'], 
         axis = 1)

In [4]:
df2 = pd.read_csv('project_data/MERGED2017_18_PP.csv')

/var/folders/vk/7c21x7qj4fv7flh8p9m2wwn00000gn/T/ipykernel_86468/539635706.py:1: DtypeWarning: Columns (1729,1909,1910,1911,1912,1913) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('project_data/MERGED2017_18_PP.csv')


In [6]:
# Manually selected some variables that seem relevant based on description
df2_slice = df2[['OPEID6','INSTNM','SCH_DEG','NUMBRANCH','PREDDEG','HIGHDEG','REGION','ADM_RATE','SATVR25','SATVR75','SATMT25',
                'SATMT75','SATWR25','SATWR75','SATVRMID','SATMTMID','SATWRMID','ACTCM25','ACTCM75','ACTEN25',
                'ACTEN75','ACTMT25','ACTMT75','ACTWR25','ACTWR75','ACTCMMID','ACTENMID','ACTMTMID','ACTWRMID',
                'SAT_AVG','UGDS','UGDS_WHITE','UGDS_BLACK','UGDS_HISP','UGDS_ASIAN','UGDS_AIAN','UGDS_NHPI',
                'UGDS_2MOR','UGDS_NRA','UGDS_UNKN','PPTUG_EF','NPT4_PUB','NPT4_PRIV','NUM4_PUB','NUM4_PRIV',
                'NUM4_PROG','NUM4_OTHER','COSTT4_A','COSTT4_P','TUITIONFEE_IN','TUITIONFEE_OUT','TUITIONFEE_PROG',
                'TUITFTE','INEXPFTE','AVGFACSAL','PFTFAC']]

# Dropped columns with only NA
df2_slice.dropna(axis=1, how='all', inplace=True)

# Use the school code to create a matching column to merge the two datasets
df2_slice['primaryKey'] = df2_slice['OPEID6']

result = pd.merge(df1, df2_slice, on='primaryKey', how="left", indicator = True)
result.shape

/var/folders/vk/7c21x7qj4fv7flh8p9m2wwn00000gn/T/ipykernel_86468/122189018.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_slice.dropna(axis=1, how='all', inplace=True)
/var/folders/vk/7c21x7qj4fv7flh8p9m2wwn00000gn/T/ipykernel_86468/122189018.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_slice['primaryKey'] = df2_slice['OPEID6']


(420, 78)

In [7]:
# Identified duplicated rows
duplicated = result[result['primaryKey'].duplicated(keep=False)]
duplicated.shape

(148, 78)

In [17]:
# Downloaded data to perform manual selection
#result.to_csv('processed_data_1.csv', index=False)
#duplicated.to_csv('duplicated_1.csv', index=False)

# Read the manually processed data
data = pd.read_csv('project_data/processed_data1.csv')

In [18]:
# Identified unranked institutions
data.loc[data.rankingDisplayRank == "Unranked"]

,act-avg,sat-avg,enrollment,city,sortName,zip,acceptance-rate,rankingDisplayScore,percent-receiving-aid,cost-after-aid,...,COSTT4_A,COSTT4_P,TUITIONFEE_IN,TUITIONFEE_OUT,TUITIONFEE_PROG,TUITFTE,INEXPFTE,AVGFACSAL,PFTFAC,_merge
300,NaN,NaN,1264.0,San Diego,alliantinternationaluniversity,92131,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
301,NaN,NaN,NaN,Orange,argosyuniversity,92868,NaN,NaN,NaN,NaN,...,29396.0,NaN,13438.0,13438.0,NaN,16793.0,6133.0,5460.0,0.1774,both
302,NaN,NaN,NaN,San Francisco,californiainstituteofintegralstudies,94103,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,22916.0,13260.0,8190.0,0.3247,both
303,NaN,NaN,NaN,Minneapolis,capellauniversity,55403,NaN,NaN,NaN,NaN,...,19836.0,NaN,14250.0,14250.0,NaN,16533.0,1714.0,6841.0,0.1387,both
304,NaN,NaN,NaN,Pocatello,idahostateuniversity,83209,NaN,NaN,NaN,NaN,...,19592.0,NaN,7166.0,21942.0,NaN,7822.0,11741.0,7156.0,0.9380,both
305,NaN,NaN,133.0,San Diego,northcentraluniversity,86314,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,16529.0,2909.0,6347.0,0.2204,left_only
306,NaN,NaN,NaN,Cypress,tridentuniversityinternational,90630,96.0,NaN,NaN,NaN,...,17544.0,NaN,9240.0,9240.0,NaN,9361.0,1828.0,5705.0,0.0614,both
307,NaN,NaN,NaN,Cincinnati,unioninstituteanduniversity,45206,NaN,NaN,NaN,NaN,...,24696.0,NaN,12896.0,12896.0,NaN,16910.0,6638.0,5540.0,0.1690,both
308,NaN,NaN,NaN,Phoenix,universityofphoenix,85034,NaN,NaN,NaN,NaN,...,20083.0,NaN,9608.0,9608.0,NaN,13180.0,2042.0,4485.0,0.0462,left_only
309,NaN,NaN,NaN,Minneapolis,waldenuniversity,55401,NaN,NaN,NaN,NaN,...,NaN,NaN,12465.0,12465.0,NaN,10183.0,2854.0,6769.0,0.0674,both


In [19]:
# Removed unranked institutions
data.drop(index = range(300,311), inplace = True)

# Dropped columns with only NA
data.dropna(axis=1, how='all', inplace=True)

# Drop columns with only 1 unique value
cols_to_drop = []
for col in data.columns:
    if data[col].nunique() == 1:
        cols_to_drop.append(col)
data.drop(cols_to_drop, axis=1, inplace = True)

In [20]:
# Combined columns that have the same information but stored separately for public and private institutions
data['NPT4'] = data['NPT4_PUB'].fillna(data['NPT4_PRIV'])
data['NUM4'] = data['NUM4_PUB'].fillna(data['NUM4_PRIV'])

# Dropped already combined columns and _merge
data.drop(['_merge','NPT4_PUB','NPT4_PRIV','NUM4_PUB','NUM4_PRIV'], axis=1, inplace=True)

In [21]:
# Create a new colume that contains SAT Average 
# Filled missing values in College Scorecard SAT average with values of SAT average from the Kaggle data 
# to minimize number of missing values
data['sat_avg'] = data['SAT_AVG'].fillna(data['sat-avg'])

# Dropped the two original columns
data.drop(['SAT_AVG','sat-avg'], axis=1, inplace=True)

In [22]:
# With the same principle, we will use enrollment data from college scorecard instead of Kaggle
# Dropped Kaggle enrollment data
data.drop(['enrollment'], axis=1, inplace=True)

In [23]:
# Rename variables to more interpretable names
data = data.rename(columns = {
    'act-avg': 'act_avg',
    'acceptance-rate': 'acceptance_rate',
    'percent-receiving-aid': 'percent_receiving_aid',
    'cost-after-aid': 'cost_after_aid',
    'hs-gpa-avg': 'hs_gpa_avg', 
    'INSTNM': 'institution_name',
    'NUMBRANCH': 'branches', 
    'REGION': 'region',
    'ADM_RATE': 'admission_rate',
    'SATVR25': 'satCR25', 
    'SATVR75': 'satCR75',
    'SATMT25': 'satmt25',
    'SATMT75': 'satmt75',
    'SATVRMID': 'satcrmid', 
    'SATMTMID': 'satmtmid',
    'ACTCM25': 'actcm25',
    'ACTCM75': 'actcm75',
    'ACTEN25': 'acten25', 
    'ACTEN75': 'acten75',
    'ACTMT25': 'actmt25',
    'ACTMT75': 'actmt75',
    'ACTCMMID': 'actcmmid', 
    'ACTENMID': 'actenmid',
    'ACTMTMID': 'actmtmid',
    'UGDS': 'enrollment', 
    'UGDS_WHITE': 'percent_white',
    'UGDS_BLACK': 'percent_black',
    'UGDS_HISP': 'percent_hispanic',
    'UGDS_ASIAN': 'percent_asian', 
    'UGDS_AIAN': 'percent_aian',
    'UGDS_NHPI': 'percent_nhpi', 
    'UGDS_2MOR': 'percent_twoormore',
    'UGDS_NRA': 'percent_nra',
    'UGDS_UNKN': 'percent_unknown', 
    'PPTUG_EF': 'percent_parttime',
    'COSTT4_A': 'avg_cost',
    'TUITIONFEE_IN': 'instante_tuition', 
    'TUITIONFEE_OUT': 'outstate_tuition',
    'TUITFTE': 'tuition_revenue_per', 
    'INEXPFTE': 'instructional_expenditure_per', 
    'AVGFACSAL': 'avg_faculty_salary', 
    'PFTFAC': 'ft_faculty_rate', 
    'NPT4': 'avg_net_price', 
    'NUM4': 'number_titleIV'
}
                  )

In [24]:
# Identified missing value in admission_rate
missing = data['admission_rate'].isna()
na_rows = data[missing]
na_rows

,act_avg,city,sortName,zip,acceptance_rate,rankingDisplayScore,percent_receiving_aid,cost_after_aid,state,rankingSortRank,...,avg_cost,instante_tuition,outstate_tuition,tuition_revenue_per,instructional_expenditure_per,avg_faculty_salary,ft_faculty_rate,avg_net_price,number_titleIV,sat_avg
263,16.0,Nashville,tennesseestateuniversity,37209,53.0,NaN,NaN,NaN,TN,-1,...,19058.0,7776.0,21132.0,6877.0,8732.0,7310.0,0.9707,11083.0,609.0,788.0


In [25]:
# Filled in missing value in College Scorecard admission rate data with Kaggle acceptance rate data
ar = data.loc[263, 'acceptance_rate']/100
data.at[263, 'admission_rate'] = ar
data.drop(['acceptance_rate'], axis=1, inplace=True)

In [ ]:
# Downloaded cleaned data
data.to_csv('cleaned_data.csv', index=False)

### Data preparation
*By Sankaranarayanan Balasubramanian and Chun-Li*

The following data preparation steps helped us to prepare our data for implementing various modeling / validation techniques:

1. Since we need to predict house price, we derived some new predictors *(from existing predictors)* that intuitively seem to be helpuful to predict house price. 

2. We have shuffled the dataset to prepare it for K-fold cross validation.

3. We have created a standardized version of the dataset, as we will use it to develop Lasso / Ridge regression models.

In [3]:
######---------------Creating new predictors----------------#########

#Creating number of bedrooms per unit floor area

#Creating ratio of bathrooms to bedrooms

#Creating ratio of carpet area to floor area

In [ ]:
######-----------Shuffling the dataset for K-fold------------#########

In [ ]:
######-----Standardizing the dataset for Lasso / Ridge-------#########

## Exploratory data analysis

Put code with comments. The comments should explain the code such that it can be easily understood. You may put text *(in a markdown cell)* before a large chunk of code to explain the overall purpose of the code, if it is not intuitive. **Put the name of the person / persons who contributed to each code chunk / set of code chunks.**

## Developing the model

Put code with comments. The comments should explain the code such that it can be easily understood. You may put text *(in a markdown cell)* before a large chunk of code to explain the overall purpose of the code, if it is not intuitive. **Put the name of the person / persons who contributed to each code chunk / set of code chunks.**

### Developing the Plain Model
*By Peggy Han*

We developed a plain model that simply uses all variables as predictors with no transformation or interaction terms included to serve as a baseline model.

In [34]:
# Plain Model
df = pd.read_csv('newdata.csv')
df = df.fillna(df.mean())

# Ranking of -1 indicates the school is ranked in the range of #231 - #300
# So I replaced them all with ranking of 300
df['rankingSortRank'] = df['rankingSortRank'].replace(-1, 300)

/var/folders/vk/7c21x7qj4fv7flh8p9m2wwn00000gn/T/ipykernel_86468/2483564295.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df = df.fillna(df.mean())


In [35]:
# Create categories for universities based on the original rankings
bins = [0, 75, 150, 225, float('inf')]
labels = ['high', 'med_high', 'med_low', 'low']
df['categories'] = pd.cut(df['rankingSortRank'], bins=bins, labels=labels)

In [28]:
X = df[['act_avg', 'percent_receiving_aid', 'cost_after_aid',
        'hs_gpa_avg', 'businessRepScore', 'tuition',
       'engineeringRepScore','branches', 'region', 'admission_rate',
       'ug_enrollment', 'percent_white', 'percent_black', 'percent_hispanic',
       'percent_asian', 'percent_aian', 'percent_nhpi', 'percent_twoormore',
       'percent_nra', 'percent_unknown', 'percent_parttime', 'avg_cost',
       'instante_tuition', 'outstate_tuition', 'tuition_revenue_per',
       'instructional_expenditure_per', 'avg_faculty_salary',
       'ft_faculty_rate', 'avg_net_price', 'number_titleIV', 'sat_avg']]

#### Using all data for train and predict on trian

In [29]:
plain_model = sm.ols(formula = 'rankingSortRank~' + '+'.join(X.columns),data = df).fit()
plain_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        rankingSortRank   R-squared:                       0.894
Model:                            OLS   Adj. R-squared:                  0.881
Method:                 Least Squares   F-statistic:                     72.63
Date:                Tue, 14 Mar 2023   Prob (F-statistic):          3.63e-112
Time:                        15:31:17   Log-Likelihood:                -1464.0
No. Observations:                 300   AIC:                             2992.
Df Residuals:                     268   BIC:                             3110.
Df Model:                          31                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                     -9885.2509   2.52e+04     -0.393      0.695   -5.95e+04    3.97e+04
act_avg                          -3.7605      1.803     -2.086      0.038      -7.309      -0.212
percent_receiving_aid             0.2265      0.249      0.911      0.363      -0.263       0.716
cost_after_aid                    0.0008      0.001      1.580      0.115      -0.000       0.002
hs_gpa_avg                      -38.3910     13.057     -2.940      0.004     -64.099     -12.683
businessRepScore                -21.4875      5.852     -3.672      0.000     -33.010      -9.965
tuition                           0.0009      0.001      0.857      0.392      -0.001       0.003
engineeringRepScore               4.3522      5.223      0.833      0.405      -5.932      14.636
branches                         -1.7699      1.304     -1.358      0.176      -4.336       0.797
region                            2.1616      1.319      1.639      0.102      -0.434       4.758
admission_rate                    7.9172     16.567      0.478      0.633     -24.700      40.535
ug_enrollment                    -0.0014      0.001     -2.659      0.008      -0.002      -0.000
percent_white                   1.05e+04   2.52e+04      0.417      0.677   -3.91e+04    6.01e+04
percent_black                  1.056e+04   2.52e+04      0.419      0.675    -3.9e+04    6.01e+04
percent_hispanic               1.051e+04   2.52e+04      0.417      0.677   -3.91e+04    6.01e+04
percent_asian                  1.046e+04   2.52e+04      0.415      0.678   -3.91e+04       6e+04
percent_aian                   9559.7019   2.52e+04      0.379      0.705   -4.01e+04    5.92e+04
percent_nhpi                   8018.1592   2.52e+04      0.319      0.750   -4.15e+04    5.76e+04
percent_twoormore              1.089e+04   2.52e+04      0.432      0.666   -3.87e+04    6.05e+04
percent_nra                    1.051e+04   2.52e+04      0.418      0.677   -3.91e+04    6.01e+04
percent_unknown                1.068e+04   2.52e+04      0.424      0.672   -3.89e+04    6.03e+04
percent_parttime                225.2990     31.535      7.144      0.000     163.211     287.387
avg_cost                         -0.0026      0.001     -2.468      0.014      -0.005      -0.001
instante_tuition                  0.0037      0.001      3.322      0.001       0.001       0.006
outstate_tuition                 -0.0029      0.001     -2.749      0.006      -0.005      -0.001
tuition_revenue_per              -0.0001      0.001     -0.180      0.858      -0.002       0.001
instructional_expenditure_per  9.454e-05      0.000      0.444      0.657      -0.000       0.001
avg_faculty_salary               -0.0058      0.002     -3.263      0.001      -0.009      -0.002
ft_faculty_rate                  -2.7051     

In [31]:
prediction = plain_model.predict(df)

In [36]:
# Create bins for universities based on the predicted rankings
bins = [-float('inf'), 75, 150, 225, float('inf')]
labels = ['high', 'med_high', 'med_low', 'low']
categories = pd.cut(prediction, bins=bins, labels=labels)
df['pred_category'] = categories

In [37]:
# Compute the accuracy using the number of matching rankings
num_matches = df['categories'].eq(df['pred_category']).value_counts(normalize=True)[True] * len(df)
num_matches/len(df)

0.82

#### Splitting into Train and Test

In [39]:
test = df.sample(n=50, random_state=1)
train = df.drop(test.index)

In [40]:
plain_model_1 = sm.ols(formula = 'rankingSortRank~' + '+'.join(X.columns),data = train).fit()
plain_model_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        rankingSortRank   R-squared:                       0.896
Model:                            OLS   Adj. R-squared:                  0.881
Method:                 Least Squares   F-statistic:                     60.41
Date:                Tue, 14 Mar 2023   Prob (F-statistic):           5.47e-90
Time:                        15:40:58   Log-Likelihood:                -1219.5
No. Observations:                 250   AIC:                             2503.
Df Residuals:                     218   BIC:                             2616.
Df Model:                          31                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                      2.129e+04    2.9e+04      0.735      0.463   -3.58e+04    7.84e+04
act_avg                          -2.9711      2.026     -1.466      0.144      -6.964       1.022
percent_receiving_aid             0.1447      0.275      0.526      0.600      -0.398       0.687
cost_after_aid                    0.0006      0.001      1.018      0.310      -0.001       0.002
hs_gpa_avg                      -33.4451     14.271     -2.344      0.020     -61.571      -5.319
businessRepScore                -18.0375      6.467     -2.789      0.006     -30.783      -5.292
tuition                           0.0008      0.001      0.732      0.465      -0.001       0.003
engineeringRepScore               5.9757      5.825      1.026      0.306      -5.504      17.455
branches                         -4.9690      2.011     -2.471      0.014      -8.933      -1.005
region                            1.3090      1.506      0.869      0.386      -1.659       4.277
admission_rate                   22.8263     18.443      1.238      0.217     -13.524      59.177
ug_enrollment                    -0.0013      0.001     -2.234      0.026      -0.002      -0.000
percent_white                 -2.069e+04    2.9e+04     -0.714      0.476   -7.78e+04    3.64e+04
percent_black                 -2.064e+04    2.9e+04     -0.712      0.477   -7.77e+04    3.65e+04
percent_hispanic              -2.067e+04    2.9e+04     -0.713      0.476   -7.78e+04    3.64e+04
percent_asian                 -2.072e+04    2.9e+04     -0.715      0.475   -7.78e+04    3.64e+04
percent_aian                   -2.16e+04    2.9e+04     -0.745      0.457   -7.87e+04    3.55e+04
percent_nhpi                  -2.303e+04    2.9e+04     -0.793      0.429   -8.03e+04    3.42e+04
percent_twoormore             -2.016e+04    2.9e+04     -0.696      0.487   -7.73e+04     3.7e+04
percent_nra                   -2.066e+04    2.9e+04     -0.713      0.476   -7.78e+04    3.64e+04
percent_unknown               -2.053e+04    2.9e+04     -0.709      0.479   -7.76e+04    3.66e+04
percent_parttime                227.4443     34.568      6.580      0.000     159.315     295.574
avg_cost                         -0.0018      0.001     -1.333      0.184      -0.004       0.001
instante_tuition                  0.0030      0.001      2.178      0.030       0.000       0.006
outstate_tuition                 -0.0028      0.001     -2.531      0.012      -0.005      -0.001
tuition_revenue_per              -0.0005      0.001     -0.599      0.550      -0.002       0.001
instructional_expenditure_per  7.235e-05      0.000      0.263      0.793      -0.000       0.001
avg_faculty_salary               -0.0069      0.002     -3.119      0.002      -0.011      -0.003
ft_faculty_rate                  -2.1890     

In [42]:
y_pred = plain_model_1.predict(test)

In [44]:
# Create bins for universities based on the predicted rankings
bins = [-float('inf'), 75, 150, 225, float('inf')]
labels = ['high', 'med_high', 'med_low', 'low']
categories = pd.cut(y_pred, bins=bins, labels=labels)
test['pred_category'] = categories

In [45]:
# Compute the accuracy using the number of matching rankings
matches = test['categories'].eq(test['pred_category']).value_counts(normalize=True)[True] * len(df)
matches/len(df)

0.78

### Code fitting the final model

Put the code(s) that fit the final model(s) in separate cell(s), i.e., the code with the `.ols()` or `.logit()` functions.

## Conclusions and Recommendations to stakeholder(s)

You may or may not have code to put in this section. Delete this section if it is irrelevant.